# Texas Tow Trucks

We're going to scrape some [tow trucks in Texas](https://www.tdlr.texas.gov/tools_search/).

## Import your imports

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import time

## Search for the TLDR Number `006179570C`, and scrape the information on that company

Using [license information system](https://www.tdlr.texas.gov/tools_search/), find information about the tow truck number above, displaying the

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

If you can't figure a 'nice' way to locate something, your two last options might be:

- **Find a "parent" element, then dig inside**
- **Find all of a type of element** (like we did with `td` before) and get the `[0]`, `[1]`, `[2]`, etc
- **XPath** (inspect an element, Copy > Copy XPath)

These kinds of techniques tend to break when you're on other result pages, but... maybe not! You won't know until you try.

> - *TIP: When you use xpath, you CANNOT use double quotes or Python will get confused. Use single quotes.*
> - *TIP: You can clean your data up if you want to, or leave it dirty to clean later*
> - *TIP: The address part can be tough, but you have a few options. You can use a combination of `.split` and list slicing to clean it now, or clean it later in the dataframe with regular expressions. Or other options, too, probably*

In [5]:
#Open chrome and go to URL
driver = webdriver.Chrome()
url = 'https://www.tdlr.texas.gov/tools_search/'
driver.get(url)
#Enter tldr number in search field
tldr = '006179570C'
driver.find_element_by_id('mcrdata').send_keys(tldr)
#clicky click click
driver.find_element_by_id('submit3').click()

In [90]:

tables = driver.find_elements_by_tag_name('tr')
name = tables[4].text
owner = tables[5].text
phone = tables[6].text
status = tables[7].text
dirty_address = tables[8].find_elements_by_tag_name('td')[1].text
clean_address = dirty_address.split(':')[4]




13619 BRETT JACKSON RD.
FORT WORTH, TX. 76179


# Adapt this to work inside of a single cell

Double-check that it works. You want it to print out all of the details.

In [103]:
tables = driver.find_elements_by_tag_name('tr')

tow_truck = {}

tow_truck['name'] = tables[4].text[8:]
tow_truck['owner'] = tables[5].text[17:]
tow_truck['phone'] = tables[6].text[9:]
tow_truck['status'] = tables[7].text[33:]
dirty_address = tables[8].find_elements_by_tag_name('td')[1].text
tow_truck['clean_address'] = dirty_address.split(':')[4]

print(tow_truck)


{'name': 'B.D. SMITH TOWING DBA:   NO DATA', 'owner': 'BRANDT SMITH / OWNER', 'phone': '8173330706', 'status': ' Expired', 'clean_address': '\n13619 BRETT JACKSON RD.\nFORT WORTH, TX. 76179'}


# Using .apply to find data about SEVERAL tow truck companies

The file `trucks-subset.csv` has information about the trucks, we'll use it to find the pages to scrape.

### Open up `trucks-subset.csv` and save it into a dataframe

In [106]:
df= pd.read_csv('trucks-subset.csv')

In [107]:
df

,TDLR Number
0,006507931C
1,006179570C
2,006502097C


0    006507931C
1    006179570C
2    006502097C
Name: TDLR Number, dtype: object

## Go through each row of the dataset, displaying the URL you will need to scrape for the information on that row

You don't have to actually use the search form for each of these - look at the URL you're on, it has the number in it!

For example, one URL might look like `https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C`.

- *TIP: Use .apply and a function*
- *TIP: Unlike the Yelp example, you'll need to build this URL from pieces*
- *TIP: You probably don't want to `print` unless you're going to fix it for the next question 
- *TIP: pandas won't showing you the entire url! Run `pd.set_option('display.max_colwidth', -1)` to display aaaalll of the text in a cell*

In [109]:
def make_url(row):
    url_base = 'https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber='
    new_url = url_base + row['TDLR Number']
    return new_url

In [114]:
pd.set_option('display.max_colwidth', -1)

In [123]:
df.apply(make_url, axis=1)

0    https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1    https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2    https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C
dtype: object

### Save this URL into a new column of your dataframe, called `url`

- *TIP: Use a function and `.apply`*
- *TIP: Be sure to use `return`*

In [124]:
df['url'] = df.apply(make_url, axis=1)

In [125]:
df

,TDLR Number,url
0,006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


## Go through each row of the dataset, printing out information about each tow truck company.

Now will be **scraping** inside of your function.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

Just print it out for now.

- *TIP: use .apply*
- *TIP: You'll be using the code you wrote before, but converted into a function*
- *TIP: Remember how the TDLR Number is in the URL? You don't need to do the form submission if you don't want!*
- *TIP: Make sure you adjust any variables so you don't scrape the same page again and again*

In [ ]:
driver = webdriver.Chrome()

In [128]:
def scraper(row):
    url = row['url']
    driver.get(url)
    
    tables = driver.find_elements_by_tag_name('tr')
    tow_truck = {}

    tow_truck['name'] = tables[4].text[8:]
    tow_truck['owner'] = tables[5].text[17:]
    tow_truck['phone'] = tables[6].text[9:]
    tow_truck['status'] = tables[7].text[33:]
    dirty_address = tables[8].find_elements_by_tag_name('td')[1].text
    tow_truck['clean_address'] = dirty_address.split(':')[4]
    print(tow_truck)

In [129]:
df.apply(scraper, axis=1)

{'name': 'AUGUSTUS E SMITH DBA:   A-1 AUTO RECOVERY', 'owner': 'AUGUSTUS EUGENE SMITH / OWNER', 'phone': '9032276464', 'status': ' Active', 'clean_address': '\n103 N MAIN ST\nBONHAM, TX. 75418'}
{'name': 'B.D. SMITH TOWING DBA:   NO DATA', 'owner': 'BRANDT SMITH / OWNER', 'phone': '8173330706', 'status': ' Expired', 'clean_address': '\n13619 BRETT JACKSON RD.\nFORT WORTH, TX. 76179'}
{'name': 'BARRY MICHAEL SMITH DBA:   BUFFALO TOWING', 'owner': 'BARRY MICHAEL SMITH / OWNER', 'phone': '8066544404', 'status': ' Active', 'clean_address': '\n4501 W CEMETERY RD\nCANYON, TX. 79015'}


0    None
1    None
2    None
dtype: object

## Scrape the following information for each row of the dataset, and save it into new columns in your dataframe.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

It's basically what we did before, but using the function a little differently.

- *TIP: Same as above, but you'll be returning a `pd.Series` and the `.apply` line is going to be a lot longer*
- *TIP: Save it to a new dataframe!*
- *TIP: Make sure you change your `df` variable names correctly if you're cutting and pasting - there are a few so it can get tricky*

In [130]:
driver = webdriver.Chrome()

In [158]:
def scraper(row):
    url = row['url']
    driver.get(url)
    
    tables = driver.find_elements_by_tag_name('tr')
    tow_truck = {}
    try:
        name = tables[4].text[8:]
    except:
        print('messed up on name')
    try:
        owner = tables[5].text[17:]
    except:
        print('messed up on owner')
    try:
        phone = tables[6].text[9:]
    except:
        print('messed up on phone')
    try:
        status = tables[7].text[33:]
    except:
        print('messed up on status')
    try:
        dirty_address = tables[8].find_elements_by_tag_name('td')[1].text
    except:
        print('messed up on dirty_address')
    try:
        clean_address = dirty_address.split(':')
    except:
        print('messed up on clean address')

    return pd.Series ({
        'name' : name,
        'owner' : owner,
        'phone' : phone,
        'status' : status,
        'clean_address' : clean_address
    })

In [139]:
small_df = df.apply(scraper, axis=1).join(df)

### Save your dataframe as a CSV named `tow-trucks-extended.csv`

In [143]:
small_df.to_csv('tow-trucks-extended.csv', index=False)

### Re-open your dataframe to confirm you didn't save any extra weird columns

In [144]:
pd.read_csv('tow-trucks-extended.csv')

,name,owner,phone,status,clean_address,TDLR Number,url
0,AUGUSTUS E SMITH DBA: A-1 AUTO RECOVERY,AUGUSTUS EUGENE SMITH / OWNER,9032276464,Active,"\n103 N MAIN ST\nBONHAM, TX. 75418",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,B.D. SMITH TOWING DBA: NO DATA,BRANDT SMITH / OWNER,8173330706,Expired,"\n13619 BRETT JACKSON RD.\nFORT WORTH, TX. 76179",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,BARRY MICHAEL SMITH DBA: BUFFALO TOWING,BARRY MICHAEL SMITH / OWNER,8066544404,Active,"\n4501 W CEMETERY RD\nCANYON, TX. 79015",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


## Process the entire `tow-trucks.csv` file

We just did it on a short subset so far. Now try it on all of the tow trucks. **Save as the same filename as before**

In [148]:
df_full = pd.read_csv('tow-trucks.csv')
df_full['url'] = df_full.apply(make_url, axis=1)
df_full.head()

,TDLR Number,url
0,006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C
3,006494912C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006494912C
4,0649468VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0649468VSF


In [160]:
df_total = df_full.apply(scraper, axis=1).join(df_full)

In [162]:
df_total.to_csv('tow-trucks.csv')